# Model Building - Loss Function

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

download pretrained network

In [2]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

feature_map = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                           include_top=False,
                           weights='imagenet')

feature_vec_base = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                           include_top=True,
                           weights='imagenet')
feature_vec_base.trainable = False
feature_map.trainable = False

Remove the top probability layer on feature vector base model

In [3]:
layer_name = "avg_pool"
feature_vec_model = tf.keras.Model(inputs=feature_vec_base.input, outputs=feature_vec_base.get_layer(layer_name).output, name="resnet50_base")

In [4]:
feature_vec_model.summary()

Model: "resnet50_base"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
______________________________________________________________________________________

In [5]:
NUM_UNITS = 200
dense_1 = tf.keras.layers.Dense(NUM_UNITS, activation="relu", name="style_embedding_1")
dense_2 = tf.keras.layers.Dense(NUM_UNITS,  name="style_embedding_2")

feature_vec_model = tf.keras.Sequential([feature_vec_model, dense_1, dense_2])

feature_vec_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50_base (Model)        (None, 2048)              23587712  
_________________________________________________________________
style_embedding_1 (Dense)    (None, 200)               409800    
_________________________________________________________________
style_embedding_2 (Dense)    (None, 200)               40200     
Total params: 24,037,712
Trainable params: 450,000
Non-trainable params: 23,587,712
_________________________________________________________________


In [6]:
len(feature_vec_model.trainable_variables)

4

In [13]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return img

In [22]:
data_path = "../data/processed/training/fashion_scene/*"
list_ds = tf.data.Dataset.list_files(str(data_path))
file_path = [i for i in list_ds.take(1)][0].numpy()
img = tf.io.read_file(file_path)
img = np.expand_dims(decode_img(img), 0)

In [23]:
feature_vec_model(img)

<tf.Tensor: shape=(1, 200), dtype=float32, numpy=
array([[ 0.2290963 , -0.97310126, -1.3362894 , -0.84249425,  0.49784186,
         0.6320254 ,  0.78999263,  0.04023098, -0.48969927,  1.0647702 ,
        -0.5138737 , -0.60160905,  0.6896185 ,  1.3664285 ,  0.46246898,
        -0.43800354,  1.0004776 ,  0.5223546 , -0.3384732 , -0.7199579 ,
         1.4581954 ,  0.0607359 ,  0.23923032, -1.0424386 ,  0.84502673,
         0.64103836, -0.67860866, -0.8866885 , -0.45321083,  1.312486  ,
         0.9553771 , -0.57179445, -1.3164413 ,  0.22674747, -0.09709118,
         1.0375495 , -0.59205365, -0.740969  ,  0.32230526,  0.3437178 ,
        -0.5676326 ,  0.61405236,  0.14641382, -0.18636307, -0.07126921,
         1.7563394 ,  0.7269984 ,  0.6902478 , -0.26070443,  0.0966002 ,
        -0.26677638, -0.31430706,  0.09590107, -0.25998485,  1.4810721 ,
        -0.1819151 ,  0.22325686, -1.5484995 ,  0.516326  ,  0.21200474,
        -0.6162795 , -0.0284168 ,  0.98019373, -0.92260385, -0.3177669 ,
 